This notebook contains functionality to perform the following:

Load in models trained using various street-lit (or not) datasets, and test on various test sets (including both accessible and inaccessible). Output .csv with summary of performance.

In [ ]:
# mount google drive

from google.colab import drive
drive.mount('/content/drive')

%cd '/content/drive/Shareddrives/NRC_Amii_Agronomics_Project/nrc-ml-plant-genomics/'

Mounted at /content/drive
/content/drive/Shareddrives/NRC_Amii_Agronomics_Project/nrc-ml-plant-genomics


In [ ]:
# imports
import argparse
import keras
import warnings, logging
import numpy as np
import pandas as pd
import datetime, time, os
import json
import random
import tensorflow as tf
import math
import glob
import ast

from keras.models import Sequential, load_model, model_from_json
from keras.layers import Input, Dense, Conv1D, MaxPooling2D, Dropout, Flatten, BatchNormalization
from tensorflow.keras.optimizers import Adam  # https://stackoverflow.com/questions/62707558/importerror-cannot-import-name-adam-from-keras-optimizers
from keras.callbacks import ModelCheckpoint, EarlyStopping  # https://machinelearningmastery.com/how-to-stop-training-deep-neural-networks-at-the-right-time-using-early-stopping/
from collections import Counter

from sklearn.metrics import r2_score, accuracy_score
from scipy.stats import spearmanr  # https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.spearmanr.html

warnings.filterwarnings('ignore')
logging.disable(1000)

# tf.random.set_seed(1202)  # https://www.tensorflow.org/api_docs/python/tf/random/set_seed
# from numpy.random import seed
# seed(1202)

random.seed(1234)

nts = ["A", "T", "C", "G"]  # list of single nucleotides
mapping = {"A": [1, 0, 0, 0], "T": [0, 0, 0, 1], "C": [0, 1, 0, 0], "G": [0, 0, 1, 0], "X":[0, 0, 0, 0]}  # cross referenced with kipoi data loader

def Spearman(y_true, y_pred):
     return (tf.py_function(spearmanr, [tf.cast(y_pred, tf.float32), 
                       tf.cast(y_true, tf.float32)], Tout = tf.float32) )

## Old 35s

In [ ]:
# define models

# simple models and complex models
def freq_model_architecture(args, in_dim):  # initializes model architecture
    mdl = Sequential()

    # this is the only layer that is enforced. to test linear regression only, set layer_1_size to 1 and layer_1_activation to "linear"
    mdl.add(Dense(args["layer_1_size"], input_dim=in_dim, activation=args["layer_1_activation"]))

    if args["layer_2_size"] > 0:       mdl.add(Dense(args["layer_2_size"], activation=args["layer_2_activation"]))
    if args["layer_3_size"] > 0:       mdl.add(Dense(args["layer_3_size"], activation=args["layer_3_activation"]))
    if args["output_layer_size"] > 0:  mdl.add(Dense(args["output_layer_size"], activation=args["output_layer_activation"]))

    return mdl

# models without linear mapping & models with linear mapping
def conv_model_architecture(args):  # initializes model architecture
    mdl = Sequential()

    conv1_train = args["conv_one_set"] != 2  # True if conv layer should be trained
    mdl.add(Conv1D(120, 5, activation='relu', input_shape=(args["input_sequence_length"], 4), name="1DConv_1", trainable=conv1_train))
    mdl.add(BatchNormalization(name="batchNorm1", trainable=conv1_train))
    mdl.add(Dropout(0.1, name="drop1"))

    conv2_train = args["conv_two_set"] != 2  # True if conv layer should be trained
    mdl.add(Conv1D(120, 5, activation='relu', name="1DConv_2", trainable=conv2_train))
    mdl.add(BatchNormalization(name="batchNorm2", trainable=conv2_train))
    mdl.add(Dropout(0.1, name="drop2"))

    if args["last_conv_layer"] == 1:  # if we are not removing last conv layer for simplicity
      conv3_train = args["conv_three_set"] != 2  # True if conv layer should be trained
      mdl.add(Conv1D(120, 5, activation='relu', name="1DConv_3", trainable=conv3_train))
      mdl.add(BatchNormalization(name="batchNorm3", trainable=conv3_train))
      mdl.add(Dropout(0.1, name="drop3"))

    mdl.add(Flatten(name="flat"))

    if args["linear_mapping"] == 1: 
        mdl.add(Dense(12, activation='linear', name="dense1", trainable=False))

    # output layer
    mdl.add(Dense(1, activation="linear", name="dense2"))

    return mdl

In [ ]:
# functions for getting data into proper format

def prepare_validation_freq(args, df):
    include = []  # captures all sequences we are including as input features

    if args["include_mononuc_freq"] == 1:  include += nts
    if args["include_dinuc_freq"] == 1:    include += [nt1+nt2 for nt1 in nts for nt2 in nts]
    if args["include_trinuc_freq"] == 1:   include += [nt1+nt2+nt3 for nt1 in nts for nt2 in nts for nt3 in nts]

    X_test = np.array(df[include])
    y_test = np.array(df["target"].tolist())

    return X_test, y_test

def prepare_validation_conv(df):
    X_test = np.array([get_ohe(sqnc) for sqnc in df["sequence"]])
    y_test = np.array(df["target"].tolist())

    return X_test, y_test

def get_ohe(sequence):  # gets sequence in format model can use (x, 4)
    return np.array([mapping[nt] for nt in sequence])

In [ ]:
# load in 35S data
thirty_five_s = pd.read_csv("data/processed/validation.csv")

nts = ["A", "T", "C", "G"]  # list of single nucleotides
include = [] + nts
include += [nt1+nt2 for nt1 in nts for nt2 in nts]
include += [nt1+nt2+nt3 for nt1 in nts for nt2 in nts for nt3 in nts]

for item in include:  # create new columns with the counts of sequences in "include"
  # print("including", item)
  thirty_five_s[item] = thirty_five_s.sequence.str.count(item)

In [ ]:
experiment_paths = []
results_on_35s_r2 = []
results_on_35s_spear = []

tester = []

for path in glob.glob("experiments/streetlight/old/*/*"):
  if "napus_models" in path:
    pass  # we don't want to do any evaluation here

  else:
    with open(path+'/settings.txt') as f:  # load in settings
      args = ast.literal_eval(f.read())

    # print(path)

    if "nucfreq" in path:  # frequency based model
      X_35s, y_35s = prepare_validation_freq(args, thirty_five_s)
      mdl = freq_model_architecture(args, X_35s.shape[1])
    else:  # conv based model
      X_35s, y_35s = prepare_validation_conv(thirty_five_s)
      mdl = conv_model_architecture(args)

    mdl.load_weights(path+'/best_model.h5')

    experiment_paths.append(path)

    y_35s_pred = mdl.predict(X_35s)
    results_on_35s_r2.append(r2_score(y_35s, y_35s_pred.reshape(1, -1)[0]))
    results_on_35s_spear.append(spearmanr(y_35s, y_35s_pred.reshape(1, -1)[0])[0])

In [ ]:
# create results dataframe
results_df = pd.DataFrame()
results_df["filepath"] = experiment_paths
results_df["r2"] = results_on_35s_r2
results_df["spearman"] = results_on_35s_spear

In [ ]:
results_df

In [ ]:
results_df.spearman.max()

0.11736842623004673

In [ ]:
# save dataframe
results_df.to_csv("experiments/streetlight/results_35s.csv", index=False)

## Accessible, Inaccessible, and 35S

In [ ]:
# define models

# simple models and complex models
def freq_model_architecture(args, in_dim):  # initializes model architecture
    mdl = Sequential()

    # this is the only layer that is enforced. to test linear regression only, set layer_1_size to 1 and layer_1_activation to "linear"
    mdl.add(Dense(args["layer_1_size"], input_dim=in_dim, activation=args["layer_1_activation"]))

    if args["layer_2_size"] > 0:       mdl.add(Dense(args["layer_2_size"], activation=args["layer_2_activation"]))
    if args["layer_3_size"] > 0:       mdl.add(Dense(args["layer_3_size"], activation=args["layer_3_activation"]))
    if args["output_layer_size"] > 0:  mdl.add(Dense(args["output_layer_size"], activation=args["output_layer_activation"]))

    return mdl

# models without linear mapping & models with linear mapping
def conv_model_architecture(args):  # initializes model architecture
    mdl = Sequential()

    conv1_train = args["conv_one_set"] != 2  # True if conv layer should be trained
    mdl.add(Conv1D(120, 5, activation='relu', input_shape=(args["input_sequence_length"], 4), name="1DConv_1", trainable=conv1_train))
    mdl.add(BatchNormalization(name="batchNorm1", trainable=conv1_train))
    mdl.add(Dropout(0.1, name="drop1"))

    conv2_train = args["conv_two_set"] != 2  # True if conv layer should be trained
    mdl.add(Conv1D(120, 5, activation='relu', name="1DConv_2", trainable=conv2_train))
    mdl.add(BatchNormalization(name="batchNorm2", trainable=conv2_train))
    mdl.add(Dropout(0.1, name="drop2"))

    if args["last_conv_layer"] == 1:  # if we are not removing last conv layer for simplicity
      conv3_train = args["conv_three_set"] != 2  # True if conv layer should be trained
      mdl.add(Conv1D(120, 5, activation='relu', name="1DConv_3", trainable=conv3_train))
      mdl.add(BatchNormalization(name="batchNorm3", trainable=conv3_train))
      mdl.add(Dropout(0.1, name="drop3"))

    mdl.add(Flatten(name="flat"))

    if args["linear_mapping"] == 1: 
        mdl.add(Dense(12, activation='linear', name="dense1", trainable=False))

    # output layer
    mdl.add(Dense(1, activation="linear", name="dense2"))

    return mdl

In [ ]:
# functions for getting data into proper format

def prepare_validation_freq(args, df):
    include = []  # captures all sequences we are including as input features

    if args["include_mononuc_freq"] == 1:  include += nts
    if args["include_dinuc_freq"] == 1:    include += [nt1+nt2 for nt1 in nts for nt2 in nts]
    if args["include_trinuc_freq"] == 1:   include += [nt1+nt2+nt3 for nt1 in nts for nt2 in nts for nt3 in nts]

    X_test = np.array(df[include])
    y_test = np.array(df["target"].tolist())

    return X_test, y_test

def prepare_validation_conv(df):
    X_test = np.array([get_ohe(sqnc) for sqnc in df["sequence"]])
    y_test = np.array(df["target"].tolist())

    return X_test, y_test

def get_ohe(sequence):  # gets sequence in format model can use (x, 4)
    return np.array([mapping[nt] for nt in sequence])


def create_freq_features(df):
  nts = ["A", "T", "C", "G"]  # list of single nucleotides
  include = [] + nts
  include += [nt1+nt2 for nt1 in nts for nt2 in nts]
  include += [nt1+nt2+nt3 for nt1 in nts for nt2 in nts for nt3 in nts]

  for item in include:  # create new columns with the counts of sequences in "include"
    # print("including", item)
    df[item] = df.sequence.str.count(item)

  return df

In [ ]:
# load in accessible data & restrict to test set

test_accessible = pd.read_csv("data/processed/athal_accessible.csv")
test_accessible = create_freq_features(test_accessible[test_accessible["set"] == "test"])

In [ ]:
# load in inaccessible data & restrict to test set

test_inaccessible = pd.read_csv("data/processed/athal_inaccessible.csv")
test_inaccessible = create_freq_features(test_inaccessible[test_inaccessible["set"] == "test"])

In [ ]:
# load in 35s

test_35s = pd.read_csv("data/processed/validation.csv")
test_35s = create_freq_features(test_35s)

In [ ]:
# define results df structure - four models on three test datasets

# columns = model_filepath, test_dataset, r2, spearman

In [ ]:
total_r2_results = []
total_spearman_results = []
experiment_paths = []
for path in glob.glob("experiments/streetlight/conclusive/models/*"):
  # get settings
  with open(path+'/settings.txt') as f:  # one model (either freq or conv)
    args = ast.literal_eval(f.read())
  
  experiment_paths.append(path.split("/")[-1])  # we should end up with four of these
  print(path.split("/")[-1])  

  # test model on sets
  model_r2_results = []
  model_spearman_results = []
  model_paths = []
  for test_set in [test_accessible, test_inaccessible, test_35s]:

    if "freq" in path:  # frequency based model
      X, y = prepare_validation_freq(args, test_set)
      mdl = freq_model_architecture(args, X.shape[1])
    else:  # conv based model
      X, y = prepare_validation_conv(test_set)
      mdl = conv_model_architecture(args)
    mdl.load_weights(path+'/best_weights.h5')

    y_pred = mdl.predict(X)
    model_r2_results.append(r2_score(y, y_pred.reshape(1, -1)[0]))

    spearman = spearmanr(y, y_pred.reshape(1, -1)[0])[0]
    print(spearman)
    model_spearman_results.append(spearman)

  total_r2_results.append(model_r2_results)
  total_spearman_results.append(model_spearman_results)

freq_20221130-234223_nuc101_lay24-64-0-1_lr0.002_bs512_accessible
698/698 [==============================] - 1s 1ms/step
0.38650482156087496
576/576 [==============================] - 1s 1ms/step
0.4335188858859176
38/38 [==============================] - 0s 1ms/step
0.0028904513912600613
freq_20221130-234316_nuc101_lay24-64-0-1_lr0.002_bs512_inaccessible
698/698 [==============================] - 1s 1ms/step
0.32456155726094166
576/576 [==============================] - 1s 1ms/step
0.5346845410150658
38/38 [==============================] - 0s 1ms/step
0.006782605863301782
conv_20221130-234403_0000_lr0.002_bs512_ep500_accessible
698/698 [==============================] - 19s 27ms/step
0.2575566480518518
576/576 [==============================] - 16s 27ms/step
0.29679510877528226
38/38 [==============================] - 1s 27ms/step
0.011810985853356874
conv_20221201-000533_0000_lr0.002_bs512_ep500_inaccessible
698/698 [==============================] - 20s 29ms/step
0.2710242113125077

In [ ]:
# create results dataframe
results_df = pd.DataFrame()

results_df["filepath"] = experiment_paths

results_df["r2_accessible"] = np.array(total_r2_results)[:,0]
results_df["r2_inaccessible"] = np.array(total_r2_results)[:,1]
results_df["r2_35s"] = np.array(total_r2_results)[:,2]

results_df["spearman_accessible"] = np.array(total_spearman_results)[:,0]
results_df["spearman_inaccessible"] = np.array(total_spearman_results)[:,1]
results_df["spearman_35s"] = np.array(total_spearman_results)[:,2]

In [ ]:
ls

 data/         'Larry Proposal.gdoc'   notebooks/   requirements.txt
 driver.ipynb   legacy/                output.csv   src/
 experiments/   models/                README.md


In [ ]:
results_df

,filepath,r2_accessible,r2_inaccessible,r2_35s,spearman_accessible,spearman_inaccessible,spearman_35s
0,freq_20221130-234223_nuc101_lay24-64-0-1_lr0.0...,0.130832,0.170505,-1.159325,0.386505,0.433519,0.002890
1,freq_20221130-234316_nuc101_lay24-64-0-1_lr0.0...,0.077866,0.272888,-0.999370,0.324562,0.534685,0.006783
2,conv_20221130-234403_0000_lr0.002_bs512_ep500_...,0.007310,0.032410,-7.860244,0.257557,0.296795,0.011811
3,conv_20221201-000533_0000_lr0.002_bs512_ep500_...,0.005903,0.127055,-2.296831,0.271024,0.404085,-0.039464


In [ ]:
results_df.to_csv("experiments/streetlight/conclusive/results.csv")